In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar100
from tensorflow.keras import layers
import shutil
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



169001437/169001437 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [3]:
model_name = 'mobilenet'
dataset = 'CIFAR100'

---

# Distillation

In [4]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1



In [5]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [6]:
teacher_model_path = '/content/base/mobilenetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha

In [7]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
teacher_model_path = '/content/base/mobilenetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 6s 8ms/step


Epoch 1:
313/313 [==============================] - 3s 7ms/step - loss: 4.3270 - accuracy: 0.0419
Test Loss: 4.3269572257995605, Test Accuracy: 0.04190000146627426
Epoch 2:
313/313 [==============================] - 2s 8ms/step - loss: 4.2136 - accuracy: 0.0831
Test Loss: 4.213561058044434, Test Accuracy: 0.08309999853372574
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 3.8014 - accuracy: 0.1375
Test Loss: 3.801384687423706, Test Accuracy: 0.13750000298023224
Epoch 4:
313/313 [==============================] - 2s 7ms/step - loss: 3.5181 - accuracy: 0.1704
Test Loss: 3.518148899078369, Test Accuracy: 0.1703999936580658
Epoch 5:
313/313 [==============================] - 2s 8ms/step - loss: 3.3738 - accuracy: 0.2004
Test Loss: 3.3737916946411133, Test Accuracy: 0.2003999948501587
Epoch 6:
313/313 [==============================] - 3s 11ms/step - loss: 3.3363 - accuracy: 0.2098
Test Loss: 3.336270809173584, Test Accuracy: 0.20980000495910645
Epoch 7:
313/313 [===

In [8]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same.zip'

# Student 0.1 (2)

In [9]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
teacher_model_path = '/content/base/mobilenetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 6ms/step
Epoch 1:
313/313 [==============================] - 3s 8ms/step - loss: 4.3619 - accuracy: 0.0328
Test Loss: 4.361875534057617, Test Accuracy: 0.03280000016093254
Epoch 2:
313/313 [==============================] - 2s 7ms/step - loss: 3.9794 - accuracy: 0.0938
Test Loss: 3.979388952255249, Test Accuracy: 0.09380000084638596
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 3.7869 - accuracy: 0.1257
Test Loss: 3.786860466003418, Test Accuracy: 0.1256999969482422
Epoch 4:
313/313 [==============================] - 2s 8ms/step - loss: 3.5835 - accuracy: 0.1621
Test Loss: 3.5835161209106445, Test Accuracy: 0.16210000216960907
Epoch 5:
313/313 [==============================] - 3s 9ms/step - loss: 3.4317 - accuracy: 0.1845
Test Loss: 3.4317452907562256, Test Accuracy: 0.18449999392032623
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 3.4489 - accuracy: 0.1862
Test Loss: 3.4489123821258545, 

In [10]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_2.zip'

#Student 0.1 (3)

In [11]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
teacher_model_path = '/content/base/mobilenetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 4s 10ms/step - loss: 4.3763 - accuracy: 0.0408
Test Loss: 4.376276969909668, Test Accuracy: 0.040800001472234726
Epoch 2:
313/313 [==============================] - 2s 6ms/step - loss: 3.9213 - accuracy: 0.1023
Test Loss: 3.921314001083374, Test Accuracy: 0.1023000031709671
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 3.6130 - accuracy: 0.1525
Test Loss: 3.6130030155181885, Test Accuracy: 0.1525000035762787
Epoch 4:
313/313 [==============================] - 3s 8ms/step - loss: 3.5522 - accuracy: 0.1720
Test Loss: 3.552229404449463, Test Accuracy: 0.1720000058412552
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 3.4813 - accuracy: 0.1911
Test Loss: 3.4812772274017334, Test Accuracy: 0.19110000133514404
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 3.3727 - accuracy: 0.2137
Test Loss: 3.3726894855499268, 

In [12]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_3", 'zip', "student_same_3")
colab_link = "/content/student_same_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_3.zip'

# Student same Alpha 0.5

In [13]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 4s 10ms/step - loss: 4.2983 - accuracy: 0.0458
Test Loss: 4.298274040222168, Test Accuracy: 0.04580000042915344
Epoch 2:
313/313 [==============================] - 3s 9ms/step - loss: 3.9157 - accuracy: 0.1044
Test Loss: 3.9157493114471436, Test Accuracy: 0.10440000146627426
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 4.0021 - accuracy: 0.1096
Test Loss: 4.002119541168213, Test Accuracy: 0.1096000000834465
Epoch 4:
313/313 [==============================] - 2s 7ms/step - loss: 3.6684 - accuracy: 0.1535
Test Loss: 3.6684441566467285, Test Accuracy: 0.1535000056028366
Epoch 5:
313/313 [==============================] - 3s 9ms/step - loss: 3.5497 - accuracy: 0.1684
Test Loss: 3.5496883392333984, Test Accuracy: 0.16840000450611115
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 3.4009 - accuracy: 0.1932
Test Loss: 3.400912046432495, 

In [14]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_50.zip'

# Student 0.5 (2)

In [15]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 3s 7ms/step - loss: 4.3915 - accuracy: 0.0286
Test Loss: 4.391459941864014, Test Accuracy: 0.028599999845027924
Epoch 2:
313/313 [==============================] - 2s 8ms/step - loss: 4.0126 - accuracy: 0.0911
Test Loss: 4.0126495361328125, Test Accuracy: 0.09109999984502792
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 3.8864 - accuracy: 0.1177
Test Loss: 3.8864030838012695, Test Accuracy: 0.1177000030875206
Epoch 4:
313/313 [==============================] - 3s 8ms/step - loss: 3.6114 - accuracy: 0.1650
Test Loss: 3.6113858222961426, Test Accuracy: 0.16500000655651093
Epoch 5:
313/313 [==============================] - 4s 12ms/step - loss: 3.5685 - accuracy: 0.1662
Test Loss: 3.5684731006622314, Test Accuracy: 0.16619999706745148
Epoch 6:
313/313 [==============================] - 2s 8ms/step - loss: 3.5473 - accuracy: 0.1816
Test Loss: 3.54729175567626

In [16]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_50_2.zip'

# Student 0.5 (3)

In [17]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 3s 8ms/step - loss: 4.3892 - accuracy: 0.0340
Test Loss: 4.389200687408447, Test Accuracy: 0.03400000184774399
Epoch 2:
313/313 [==============================] - 3s 10ms/step - loss: 3.9103 - accuracy: 0.1063
Test Loss: 3.910266876220703, Test Accuracy: 0.1062999963760376
Epoch 3:
313/313 [==============================] - 3s 8ms/step - loss: 3.7025 - accuracy: 0.1359
Test Loss: 3.7024996280670166, Test Accuracy: 0.13590000569820404
Epoch 4:
313/313 [==============================] - 3s 8ms/step - loss: 3.5570 - accuracy: 0.1623
Test Loss: 3.556993246078491, Test Accuracy: 0.1623000055551529
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 3.3929 - accuracy: 0.1925
Test Loss: 3.392906665802002, Test Accuracy: 0.19249999523162842
Epoch 6:
313/313 [==============================] - 3s 9ms/step - loss: 3.3724 - accuracy: 0.2012
Test Loss: 3.3724100589752197, T

In [18]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_50_3.zip'

# Student same Alpha 0.9

In [19]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 3s 8ms/step - loss: 4.4123 - accuracy: 0.0388
Test Loss: 4.412301063537598, Test Accuracy: 0.03880000114440918
Epoch 2:
313/313 [==============================] - 3s 9ms/step - loss: 3.9702 - accuracy: 0.0924
Test Loss: 3.9702465534210205, Test Accuracy: 0.09239999949932098
Epoch 3:
313/313 [==============================] - 2s 8ms/step - loss: 3.7315 - accuracy: 0.1352
Test Loss: 3.7315456867218018, Test Accuracy: 0.13519999384880066
Epoch 4:
313/313 [==============================] - 3s 10ms/step - loss: 3.5053 - accuracy: 0.1703
Test Loss: 3.5053117275238037, Test Accuracy: 0.17030000686645508
Epoch 5:
313/313 [==============================] - 3s 8ms/step - loss: 3.3650 - accuracy: 0.2011
Test Loss: 3.365046501159668, Test Accuracy: 0.20110000669956207
Epoch 6:
313/313 [==============================] - 2s 8ms/step - loss: 3.5549 - accuracy: 0.1877
Test Loss: 3.554919481277466

In [20]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_90.zip'

# Student 0.9 (2)

In [21]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 2s 5ms/step
Epoch 1:
313/313 [==============================] - 3s 8ms/step - loss: 4.2863 - accuracy: 0.0431
Test Loss: 4.286263465881348, Test Accuracy: 0.04309999942779541
Epoch 2:
313/313 [==============================] - 2s 7ms/step - loss: 4.1069 - accuracy: 0.0840
Test Loss: 4.106893539428711, Test Accuracy: 0.08399999886751175
Epoch 3:
313/313 [==============================] - 2s 8ms/step - loss: 3.8566 - accuracy: 0.1202
Test Loss: 3.856600761413574, Test Accuracy: 0.12020000070333481
Epoch 4:
313/313 [==============================] - 2s 8ms/step - loss: 3.6163 - accuracy: 0.1533
Test Loss: 3.6163313388824463, Test Accuracy: 0.15330000221729279
Epoch 5:
313/313 [==============================] - 2s 7ms/step - loss: 3.4974 - accuracy: 0.1718
Test Loss: 3.4973855018615723, Test Accuracy: 0.17180000245571136
Epoch 6:
313/313 [==============================] - 3s 10ms/step - loss: 3.4963 - accuracy: 0.1854
Test Loss: 3.4962809085845947

In [22]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_90_2.zip'

# Student 0.9 (3)

In [23]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/mobilenetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 9ms/step
Epoch 1:
313/313 [==============================] - 4s 10ms/step - loss: 4.3616 - accuracy: 0.0432
Test Loss: 4.361577033996582, Test Accuracy: 0.04320000112056732
Epoch 2:
313/313 [==============================] - 3s 8ms/step - loss: 3.8318 - accuracy: 0.1137
Test Loss: 3.8318445682525635, Test Accuracy: 0.1137000024318695
Epoch 3:
313/313 [==============================] - 2s 7ms/step - loss: 3.6791 - accuracy: 0.1411
Test Loss: 3.679089069366455, Test Accuracy: 0.14110000431537628
Epoch 4:
313/313 [==============================] - 3s 11ms/step - loss: 3.4811 - accuracy: 0.1748
Test Loss: 3.4811480045318604, Test Accuracy: 0.17479999363422394
Epoch 5:
313/313 [==============================] - 3s 9ms/step - loss: 3.4476 - accuracy: 0.1894
Test Loss: 3.447633981704712, Test Accuracy: 0.18940000236034393
Epoch 6:
313/313 [==============================] - 3s 8ms/step - loss: 3.3751 - accuracy: 0.2027
Test Loss: 3.375054359436035,

In [24]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/MobNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/MobNet_D_C100/student_same_90_3.zip'

---